<div style="color:orange; text-shadow: 1px 1px 2px #000000; font-weight:bold; font-size:30px; text-align:center;">
    Warehouse Workload Log Analysis
</div>

<hr style="border:1px solid black;">

This analysis focuses on evaluating a **hand-logged dataset** detailing daily warehouse activities, particularly emphasizing workloads related to **batch, inbound, and outbound processes**. Envisioned as a vast dataset, the log will be processed through an **ETL (Extract, Transform, Load) pipeline** to efficiently manage and handle the large volume of data for optimal analysis.

The goal of the analysis is to **identify trends and patterns** in warehouse operations, providing a deeper understanding of how each workflow—batching, inbound, and outbound—affects overall performance. Key areas of focus will include workload distribution, time variations across tasks, and the interconnectedness of different operations. By uncovering these insights, the analysis aims to inform decisions that can **optimize warehouse efficiency**, improve task coordination, and enhance resource management.

The insights derived will support **data-driven strategies** to address workload imbalances and improve operational flow, ultimately boosting warehouse productivity.

<div style="background-color:#000080; color:white; padding:10px; font-size:20px;">
    1. Extract Data by Date and Organize into a List of Dictionaries
</div

<div style="color:teal;font-size:16px;font-weight:bold">
   1.1 Library Imports
</div>

In [ ]:
import os
import re
from datetime import datetime
import pandas as pd
from pprint import pprint 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import seaborn as sns


# Get the current working directory
current_path = os.getcwd()
print(current_path)

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<div style="color:teal;font-size:16px;font-weight:bold">
   1.2 Develop Functions for Parsing and Formatting Data
</div>

##### 1.2.1 Time Range Formatting Function

In [ ]:
def parse_time(time_str):
    """Parse and format the time range to ensure two-digit hours and minutes."""
    try:
        start_time, end_time = time_str.split('-')
        return f"{datetime.strptime(start_time.strip(), '%H:%M').strftime('%H:%M')}-" \
               f"{datetime.strptime(end_time.strip(), '%H:%M').strftime('%H:%M')}"
    except ValueError:
        return time_str  # Return original format if parsing fails

##### 1.2.2 Label Categorization Function

In [ ]:
def categorize_label(label):
    """Categorize based on label prefix."""
    #label = re.sub(r'[\s-]', '_', label.strip().upper())  # Format label for consistency
    if label.startswith('IB_'):
        return 'Inbound'
    elif label.startswith('OB_'):
        return 'Outbound'
    return label.title()  # Use label as category if no specific prefix

##### 1.2.3 Regex-Based Split Function

In [ ]:
def split_with_regex(item):
    """
    Applies a regex pattern to extract specific groups from a given string.

    This function uses a predefined regex pattern to match and extract four groups from the input string:
    - 'label': The main text label before the equal sign ('=').
    - 'Numbers': A list of integers extracted after the equal sign ('=').
    - 'Time': A time range in the format 'hh:mm-hh:mm', standardized to two-digit hours and minutes.
    - 'Notes': Optional descriptive notes starting with an asterisk ('*').
    - 'Weight': Optional weigth starting with comma or equal sign(',' or '=')

    Parameters:
    ----------
    item : str
        The input string to be parsed by the regex pattern.

    Returns:
    -------
    dict or None
        A dictionary containing the parsed groups if the input matches the pattern, with keys:
        - 'Label': str, the main label extracted from the string.
        - 'Numbers': list of int, the cleaned numbers extracted from the string.
        - 'Time': str, the standardized time range with two-digit hours and minutes.
        - 'Notes': str, any additional notes extracted from the string.
        - 'Weight': int, extract from the string.
        - 'Category': str, extract from Label based on the prefix
        Returns None if the input does not match the regex pattern.
    """
    match_obj = pattern.match(item)
    if not match_obj:
        return None

    label = re.sub(r'[\s*-]', '_', match_obj.group(1).strip().upper())  # Formatted Label, extracted from the string
    category = categorize_label(label)  # Categorize the label
    numbers = [int(num) for num in re.findall(r'\d+', match_obj.group(2))]
    time = parse_time(match_obj.group(3)) if match_obj.group(3) else None
    notes = match_obj.group(4).strip() if match_obj.group(4) else None
    weight = int(match_obj.group(5)) if match_obj.group(5) else None  # Convert weight to integer

    return {
        'Label': label,
        'Category': category,
        'Numbers': numbers,
        'Time': time,
        'Notes': notes,
        'Weight': weight
    }

##### 1.2.4 Parse Data Line-by-Line into a List of Dictionaries

In [ ]:
# File path
file_path = r"/kaggle/input/grocery-warehouse-task-log-dataset/gw_task_log.txt"

# Initialize list to store parsed data as dictionaries
parsed_data = []

# Define a regex pattern to identify date lines (case-insensitive)
date_pattern = re.compile(r'^[A-Za-z]+\s*\d{1,2}\s*$', re.IGNORECASE)

# Define the regex pattern
pattern = re.compile(
    r'([A-Za-z\s_-]+)\s*=\s*([\d+\s+]+)'  # Category and Numbers
    r'(?:\s*@\s*(\d{1,2}:\d{1,2}\s*-\s*\d{1,2}:\d{1,2}))?'  # Optional Time Range
    r'(?:\s*\*([^=]*))?'  # Optional Note, non-greedy, up to a comma
    r'(?:[=]\s*(\d+)?\s*[lL][bB][sS])?' 
)

# Read the file line-by-line and parse data by date intervals
with open(file_path, "r") as file:
    current_date = ""
    for line in file:
        line = line.strip()
        # Check if the line matches a date pattern
        if date_pattern.match(line):
            current_date = line.title()  # Set the current date with proper capitalization
        elif current_date:
            # Parse the line into components using split_with_regex
            parsed_entry = split_with_regex(line)
            if parsed_entry:  # Check if parsed_entry is not None before proceeding
                parsed_entry['Date'] = current_date + ', 2024'  # Add the current date to the parsed entry
                parsed_data.append(parsed_entry)  # Append only valid entries

# Display the parsed data in a readable format
pprint(parsed_data)

<div style="color:teal;font-size:16px;font-weight:bold">
   1.3 Convert to DataFrame and Perform Post-Cleanup
</div>

##### 1.3.1 DataFrame Conversion and Verification

In [ ]:
# Convert the list of dictionaries into a DataFrame for easier data manipulation
df = pd.DataFrame(parsed_data)
print(df.info())
# Display the last 10 rows of the DataFrame to verify the calculations and the dataframe's final state
print (df.tail(3))

##### 1.3.2 Column Addition, Modification, and Data Type Adjustment

In [ ]:
# Convert column names to captialize e using .str.title()
df.columns = df.columns.str.title()

# Calculate the sum of the list of integers in each row of the 'Numbers' column and store it in a new column 'Num_Sum'
df['Num_Sum'] = df['Numbers'].apply(sum)

# Calculate the total count of integers in each list within the 'Numbers' column and store it in a new column 'Num_Count'
df['Num_Count'] = df['Numbers'].apply(len)

# Convert 'date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

df.tail()

<div style="background-color:#000080; color:white; padding:10px; font-size:20px;">
    2. Exploratory Data Analysis (EDA)
</div

In [ ]:
# Subset the data
df_sub = df[df['Category'].isin(['Inbound', 'Outbound', 'Batch'])]

# Group by 'Date' and 'Category' and aggregate the sum of 'Num_Sum' and 'Weight'
df_grouped = df_sub.groupby(['Date', 'Category'])[['Num_Sum', 'Weight']].sum().reset_index()
df_grouped.head(5)

In [ ]:
# Create a box plot for 'Num_Sum' across the 'Category'
plt.figure(figsize=(10, 4))
sns.boxplot(x='Category', y='Num_Sum', data=df_grouped)
plt.title('Daily Volume by Category')

# Add a custom Y-axis label
plt.ylabel('Volume')

plt.show()

<div style="background-color:#000080; color:white; padding:10px; font-size:20px;">
    3. Visualize Volume and Weight by Date and Category
</div

<div style="color:teal;font-size:16px;font-weight:bold">
   3.1 Preparation for Subplot 1
</div>


##### 3.1.1 Assign to a New Variable

In [ ]:
# Re-Assign to new variable
result_df = df_grouped
result_df.tail(3)

##### 3.1.2 Create a Pivot Table to Display Volume and Weight by Date and Category

In [ ]:
# Pivot the table to show num_sum and weight by date and category
pivot_table = result_df.pivot(index='Date', columns='Category', values=['Num_Sum', 'Weight'])

# Display the pivoted DataFrame
pivot_table.head(3)

<div style="color:teal;font-size:16px;font-weight:bold">
   3.2 Preparation for Subplot 2: Adding Duration: Total Work Hours
</div>

In [ ]:
# Define the regex pattern
#time_pattern = r'[e|s]os\s*\d{1,2}:\d{1,2}\s*'
time_pattern = r'(eos|sos)\s*\d{1,2}:\d{1,2}\s*'

# Create df_time by directly from subbetting DFl
df_time = df[['Date', 'Notes']].copy()

# Use regex to extract the service times
df_time['Service'] = df_time['Notes'].apply(
    lambda x: re.search(time_pattern, str(x).lower()).group(0) if pd.notnull(x) and re.search(time_pattern, str(x).lower()) else None)

# Filter rows where 'Service' is not null
df_time = df_time[['Date', 'Service']][df_time['Service'].notnull()]


# Split 'Service' into 'Type' (sos or eos) and 'Clock' (time)
df_time[['Type', 'Clock']] = df_time['Service'].str.extract(r'(eos|sos)\s*(\d{1,2}:\d{1,2})')


# Pivot the table
pivot_df_time = df_time.pivot_table(index='Date', columns='Type', values='Clock', aggfunc='first').reset_index()

# Combine 'Date' with 'sos' and 'eos' to create full datetime columns
pivot_df_time['sos_full'] = pd.to_datetime(pivot_df_time['Date'].astype(str) + ' ' + pivot_df_time['sos'].astype(str))
pivot_df_time['eos_full'] = pd.to_datetime(pivot_df_time['Date'].astype(str) + ' ' + pivot_df_time['eos'].astype(str))

# Calculate duration in hours
pivot_df_time['Duration'] = ((pivot_df_time['eos_full'] - pivot_df_time['sos_full']).dt.total_seconds() / 3600).round(2)

# Subtract 0.5 hours if duration exceeds 6 hours
pivot_df_time['Duration'] = np.where(pivot_df_time['Duration'] > 6, pivot_df_time['Duration'] - 0.5, pivot_df_time['Duration'])

# Display the final DataFrame
pivot_df_time[['Date', 'sos', 'eos', 'Duration']].head(2)

<div style="color:teal;font-size:16px;font-weight:bold">
   3.3 Plotting
</div>

In [ ]:
import matplotlib.pyplot as plt

# Create a figure with 2 subplots, adjusting height ratios
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), gridspec_kw={'height_ratios': [3, 1]})

# Plot 1: num_sum for Batch, Inbound, and Outbound on the primary y-axis (ax1)
pivot_table['Num_Sum'][['Batch', 'Inbound', 'Outbound']].plot(kind='line', ax=ax1, marker='o')
ax1.set_xlabel('Date')
ax1.set_ylabel('Volume')
ax1.set_title('Volume and Weight by Date by Category', color='teal')
ax1.grid(True)

# Set x-ticks and labels using the actual date index
ax1.set_xticks(pivot_table.index)
ax1.set_xticklabels(pivot_table.index.strftime('%Y-%m-%d'), fontsize=10, rotation=45, ha='right')
#ax1.set_xticklabels([])

# Plot 2:  weight for Inbound on the secondary y-axis (ax1)
ax1_2 = ax1.twinx()
pivot_table['Weight']['Inbound'].plot(kind='line', ax=ax1_2, marker='x', color='red')
ax1_2.set_ylabel('Weight (lb)')

# Add legends
ax1.legend(loc='upper left', fontsize=8, title_fontsize=9, title='Volume by Category')
ax1_2.legend(['Weight (Inbound)'], loc='upper center', fontsize='small')

# Plot Duration over Date on the second subplot (ax2)
ax2.plot(pivot_df_time['Date'], pivot_df_time['Duration'], marker='o', color='brown')
ax2.set_title('Total Work Hours by Date', color='teal')
#ax2.set_xlabel('Date')
ax2.set_ylabel('Duration (hr)')
ax2.set_xticks(pivot_df_time['Date'])
#ax2.set_xticklabels(pivot_df_time['Date'].dt.strftime('%Y-%m-%d'), fontsize = 10, rotation=45)
ax2.set_xticklabels([])
ax2.grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plots
plt.show()

<div style="background-color:#000080; color:white; padding:10px; font-size:20px;">
    4. Analysis of Trends and Patterns in Warehouse Workload
</div
    
1. **Weekly Seasonal Pattern**:
   - The data reveals a **weekly trend** where the workload is more evenly distributed on **Thursdays**, indicating a balance between inbound and outbound processes, as well as batching operations. However, there is a noticeable **spike in inbound activity on Fridays**, leading to heavier workloads for incoming goods. This pattern suggests that Fridays require more resources or attention for inbound operations, while Thursdays represent a more stable equilibrium across all processes.

2. **Impact of Balanced Workload**:
   - When the workload is **more balanced across different operations**, it results in a **decrease in the overall volume** being processed. This suggests that when batch, inbound, and outbound processes are evenly spread, the workload is more manageable, which might slow down the pace of operations but allows for a more steady and sustainable workflow. This balance could lead to fewer bottlenecks and reduce the pressure on specific areas, ultimately contributing to improved efficiency across the warehouse..